In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Initial population - continous problems

In [ ]:
# Pseudo-random
# generate 200 random numbers
np.random.seed(6345245)
P_random_pseudo=np.random.rand(200,200)

In [ ]:
# Quasi-random
P_random_quasi = np.random.uniform(size=(100*2))

In [ ]:
# Sobol low-discrepancy-sequence (LDS)
import sobol_seq
P_sobel=sobol_seq.i4_sobol_generate(2,200)

In [ ]:
# Latin Hypercube sampling\
import lhsmdu
num = 200
P_LHS = lhsmdu.sample(2, num)

In [ ]:
f, (ax1, ax2, ax3,ax4) = plt.subplots(ncols=4, figsize=(18,7))
ax1.scatter(P_random_pseudo[:,0], P_random_pseudo[:,1], color="gray")
ax2.scatter(P_random_quasi[:100], P_random_quasi[100:], color="red")
ax3.scatter(P_sobel[:,0], P_sobel[:,1], color="green")
ax4.plot(P_LHS[:, 0], P_LHS[:, 1], "o")

ax1.set_title("Pseudo-random")
ax2.set_title("Quasi-random")
ax3.set_title("Sobol")
ax4.set_title("Latin Hypercube")

# Initial population - permutations

In [ ]:
# randomly permute a sequence, or return a permuted range.
per1=np.random.permutation(10)
print(per1)

In [ ]:
# another method
per2 = np.array([5, 4, 9, 0, 1, 2, 6, 8, 7, 3])
np.random.shuffle(per2)
print(per2)

In [ ]:
# population of initial solution as real-value permuations 
pop_init = np.arange(50).reshape((10,5))
np.random.permutation(pop_init)

In [ ]:
# population of initial solution as binary permuations 
from itertools import combinations
size=5 # number of bits in the binary string
ones=2 # number of ones in each binary string

for pos in map(set, combinations(range(size), ones)):
     print([int(i in pos) for i in range(size)], sep='\n')

# Initial population - road routes

In [ ]:
import osmnx as ox
import random
from collections import deque

In [ ]:
G = ox.graph_from_place("University of Toronto")

In [ ]:
fig, ax = ox.plot_graph(G)

In [ ]:
#
# This class is just a wrapper around networkx graph nodes to ease its usage
# 

class Node:
    # using __slots__ for optimization
    __slots__ = ['node', 'distance', 'parent', 'osmid', 'G']
    # constructor for each node
    def __init__(self ,graph , osmid, distance = 0, parent = None):
        # the dictionary of each node as in networkx graph --- still needed for internal usage
        self.node = graph[osmid]
        
        # the distance from the parent node --- edge length
        self.distance = distance
        
        # the parent node
        self.parent = parent
        
        # unique identifier for each node so we don't use the dictionary returned from osmnx
        self.osmid = osmid
        
        # the graph
        self.G = graph
    
    # returning all the nodes adjacent to the node
    def expand(self):
        children = [Node(graph = self.G, osmid = child, distance = self.node[child][0]['length'], parent = self) \
                        for child in self.node]
        return children
    
    # returns the path from that node to the origin as a list and the length of that path
    def path(self):
        node = self
        path = []
        while node:
            path.append(node.osmid)
            node = node.parent
        return path[::-1]
    
    # the following two methods are for dictating how comparison works

    def __eq__(self, other):
        try:
            return self.osmid == other.osmid
        except:
            return self.osmid == other
            
    
    def __hash__(self):
        return hash(self.osmid)


In [ ]:
# this is just a typical graph search with suffled frontier
def randomized_search(G, source, destination):
    origin = Node(graph = G, osmid = source)
    destination = Node(graph = G, osmid = destination)
    
    route = [] 
    frontier = deque([origin])
    explored = set()
    while frontier:
        node = random.choice(frontier)   # here is the randomization part
        frontier.remove(node)
        explored.add(node.osmid)

        for child in node.expand():
            if child not in explored and child not in frontier:
                if child == destination:
                    route = child.path()
                    return route
                frontier.append(child)

    raise Exception("destination and source are not on same component")

In [ ]:
# generate random route between 2 nodes 
random_route = randomized_search(G, 24959528, 1480794706)

In [ ]:
fig, ax = ox.plot_graph_route(G, random_route)

In [ ]:
random_hexa = lambda: random.randint(0,255) # generate random hexadecimal color

In [ ]:
# generate 5 random routes with 5 different colors -- overlapping routes cancel each other color's
routes = [randomized_search(G, 24959528, 1480794706) for _ in range(5)]
rc = ['#%02X%02X%02X' % (random_hexa(),random_hexa(),random_hexa()) for _ in range(5)]
fig, ax = ox.plot_graph_routes(G, routes, route_colors=rc, route_linewidth=6, node_size=0)